# Speckle Automate notebook example

This is an example jupyter notebook, that will be executed by speckle automate

## Inputs

The first two block block are defining two input data structures, `SpeckleProjectData` and `FunctionInputs`.
The project data class is following the data schema of Automate, you shouldn't modify it.


In [7]:
from pydantic import BaseModel, ConfigDict
from stringcase import camelcase


class SpeckleProjectData(BaseModel):
    """
    Only modify this to follow upstream changes in automate!

    Values of the project / model that triggered the run of this function.
    """

    project_id: str
    model_id: str
    version_id: str
    speckle_server_url: str

    model_config = ConfigDict(alias_generator=camelcase, protected_namespaces=())

# Parameters

These values are supplied to the notebook, when its being run by automate.

WANING: You really shouldn't modify this block, unless its following upstream changes from automate.


In [8]:
speckle_project_data = '{"projectId": "03434ee1f1", "versionId": "09d2a0e55a", "modelId": "base design", "speckleServerUrl": "https://latest.speckle.dev" }'
function_inputs = '{"speckleType": "Objects.Geometry.Brep"}'
token_env_var = "SPECKLE_TOKEN"

# Function inputs

The `FunctionInputs` class defines the schema for the values, this function requires to run.
These values will be provided by the users of the function.

Automate uses the Json Schema, generated from the class, to validate user provided values.

The schema block is tagget with the `function_input` tag, do not remove that!


In [9]:
from pydantic import BaseModel, ConfigDict
from stringcase import camelcase
import json


class FunctionInputs(BaseModel):
    """User defined inputs to the function."""

    speckle_type: str

    model_config = ConfigDict(alias_generator=camelcase)


print(json.dumps(FunctionInputs.model_json_schema()))

{"description": "User defined inputs to the function.", "properties": {"speckleType": {"title": "Speckletype", "type": "string"}}, "required": ["speckleType"], "title": "FunctionInputs", "type": "object"}


# Function logic

in this block we're defining the actual business logic of the function.

By all means modify this block but do keep the function signature compatible with the executing main function.


In [10]:
from specklepy.api.client import SpeckleClient
from specklepy.objects import Base
from specklepy.transports.memory import MemoryTransport
from specklepy.transports.server import ServerTransport
from specklepy.api.operations import receive
from typing import Iterable


def flatten_base(base: Base) -> Iterable[Base]:
    if hasattr(base, "elements"):
        for element in base.elements:
            yield from flatten_base(element)
    yield base


def automate_function(
    project_data: SpeckleProjectData,
    function_inputs: FunctionInputs,
    speckle_token: str,
):
    client = SpeckleClient(project_data.speckle_server_url)
    client.authenticate_with_token(speckle_token)
    commit = client.commit.get(project_data.project_id, project_data.version_id)

    memory_transport = MemoryTransport()
    server_transport = ServerTransport(project_data.project_id, client)
    base = receive(commit.referencedObject, server_transport, memory_transport)

    matching_types = [
        b for b in flatten_base(base) if b.speckle_type == function_inputs.speckle_type
    ]
    print(f"Found {len(matching_types)} elements that have the speckle_type {function_inputs.speckle_type}")

In [11]:
import os


def main():
    inputs = FunctionInputs.model_validate_json(function_inputs)
    project_data = SpeckleProjectData.model_validate_json(speckle_project_data)
    speckle_token = os.environ[token_env_var]
    print(project_data)
    print(inputs)

    automate_function(project_data, inputs, speckle_token)
    
main()